In [35]:
import pandas as pd
import os
tripstats_csv = os.path.join('trip_stats.csv')
tripareas_csv = os.path.join('trip_areas.csv')
tripstats_df = pd.read_csv(tripstats_csv)
tripareas_df = pd.read_csv(tripareas_csv)

In [36]:
tripstats_df.columns

Index(['trip_area', 'day_of_week', 'hour', 'pickups', 'dropoffs'], dtype='object')

In [37]:
# tripstats_df.head()

In [38]:
# tripareas_df.head()

In [26]:
tripstats_df.groupby(['trip_area','day_of_week','hour']).sum()
tripstats_df.head()

,trip_area,day_of_week,hour,pickups,dropoffs
0,1,0,3,0.6,0.4
1,1,0,4,0.0,0.4
2,1,0,5,0.6,0.6
3,1,0,6,0.8,1.0
4,1,0,7,0.6,0.8


In [24]:
trip_area_df = tripstats_df.groupby(['trip_area']).sum()
trip_area_df['pickdrop'] = trip_area_df['pickups'] + trip_area_df['dropoffs']
trip_area_df['pickdrop_scaled']=trip_area_df['pickdrop']/20
trip_area_df['pickups_scaled']=trip_area_df['pickups']/20
trip_area_df['dropoffs_scaled']=trip_area_df['dropoffs']/20
trip_area_df = trip_area_df.reset_index()
# trip_area_df.head()

In [8]:
weeklytrip_coordinates = pd.merge(trip_area_df, tripareas_df, on='trip_area')

In [23]:
# weeklytrip_coordinates = weeklytrip_coordinates[['trip_area', 'pickdrop', 'latitude', 'longitude', 'pickups', 'dropoffs']]
# weeklytrip_coordinates.head()

In [11]:
output = os.path.join('avg_weeklytrip.csv')
weeklytrip_coordinates.to_csv(output, header=True)

In [12]:
trip_hours = tripstats_df.groupby(['trip_area','hour']).sum().reset_index()
trip_hours = trip_hours[['trip_area', 'hour', 'pickups', 'dropoffs']]
trip_hours['pickdrop'] = trip_hours['pickups'] + trip_hours['dropoffs']

In [13]:
trip_hours.head()

,trip_area,hour,pickups,dropoffs,pickdrop
0,1,3,0.85,1.65,2.50
1,1,4,1.00,1.10,2.10
2,1,5,3.65,4.75,8.40
3,1,6,4.20,4.35,8.55
4,1,7,4.45,4.75,9.20


In [22]:
# trip_hours.loc[1]

In [21]:
# trip_hours.loc[trip_hours['trip_area']==1]

In [39]:
trip_area_stats_df = pd.merge(tripareas_df, tripstats_df, on='trip_area')
trip_area_stats_df.head()

,trip_area,latitude,longitude,name,day_of_week,hour,pickups,dropoffs
0,1,37.708329,-122.41533,TAZ981999,0,3,0.6,0.4
1,1,37.708329,-122.41533,TAZ981999,0,4,0.0,0.4
2,1,37.708329,-122.41533,TAZ981999,0,5,0.6,0.6
3,1,37.708329,-122.41533,TAZ981999,0,6,0.8,1.0
4,1,37.708329,-122.41533,TAZ981999,0,7,0.6,0.8


In [40]:
output = os.path.join('uber_lyft_DB.csv')
trip_area_stats_df.to_csv(output, header=True)

In [42]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.orm import Session

In [43]:
Base = declarative_base()
engine = create_engine('sqlite:///uber_lyft_DB.sqlite')

In [44]:
class Rideshare(Base):
    __tablename__='rideshare'
    id = Column(Integer, primary_key=True)
    trip_area = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)
    name = Column(String(255))
    day_of_week = Column(Integer)
    hour = Column(Integer)
    pickups = Column(Float)
    dropoffs = Column(Float)

In [45]:
Base.metadata.create_all(engine)
session = Session(bind=engine)